In [3]:
import numpy as np
import pandas as pd
%matplotlib inline

from google.colab import drive 
drive.mount('/content/gdrive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/gdrive


In [0]:
OUL = 10 # Order Upto Level
ROL = 5 # Re-Order Level
alpha = 0.3

In [6]:
sales = pd.read_csv('/content/gdrive/My Drive/A Reinforcement Learning Approach for Inventory Optimization in Retail/DemandRestoring_v0/data/sales_4001.csv', sep=',', decimal='.')
sales['Timestamp'] = pd.to_datetime(sales[u'purchase_date'])
sales = sales.set_index(['Timestamp'], drop=True)
sales.head()

,purchase_date,shop_id,product_id,price,quantity,price_after_discount
Timestamp,,,,,,
2013-01-02,2013-01-02,4001,4100024369,1480,1,1480.0
2013-01-02,2013-01-02,4001,4100024794,2807,1,2807.0
2013-01-02,2013-01-02,4001,4100028657,979,1,979.0
2013-01-02,2013-01-02,4001,4300000205,3319,1,3319.0
2013-01-02,2013-01-02,4001,4300007581,1899,1,1899.0


In [7]:
Products = pd.read_csv('/content/gdrive/My Drive/A Reinforcement Learning Approach for Inventory Optimization in Retail/DemandRestoring_v0/data/Products.csv', sep=';', decimal='.', low_memory=False)
Products.head()

,Материал,Наим товара,Уровень 1 (ТН новая),Уровень 2 (Категория),Выкладка (иерарх),Дизайн (иерарх),Материал изготов-я (иерарх),Подкатегория (иерарх),Проба (иерарх),Стиль (иерарх),Тип ношения (иерарх),Ценовой сегмент (иерарх),Коллекция код,Коллекция название,"Вес нетто, г","Вес брутто, г",Основная вставка,Цвет металла,Размер цепей браслетов,Размер колец подвесов Конго
0,4300019891,Кольцо родир. алм.гр. 15.0,ИФБК,CLASSIC CZ NS,(Не определен),УЗОРЫ,(Не определен),(Не определен),585,КЛАССИКА,КОЛЬЦО,СРЕДНЯЯ,NaN,(Не определен),"1,45","1,45",NaN,КРАСНЫЙ,0,15
1,4300019831,Кольцо родир. алм.гр. 16.0,ИФБК,CLASSIC CZ NS,(Не определен),ГЕОМЕТРИЯ,(Не определен),(Не определен),585,КЛАССИКА,КОЛЬЦО,СРЕДНЯЯ,NaN,(Не определен),"1,513","1,513",NaN,КРАСНЫЙ,0,16
2,4300019887,Кольцо родир. алм.гр. 16.0,ИФБК,CLASSIC CZ NS,(Не определен),УЗОРЫ,(Не определен),(Не определен),585,КЛАССИКА,КОЛЬЦО,СРЕДНЯЯ,NaN,(Не определен),"1,546","1,546",NaN,КРАСНЫЙ,0,16
3,4300019860,Кольцо родир. алм.гр. 17.0,ИФБК,CLASSIC CZ NS,(Не определен),ЛЕПЕСТ_ПЕТ,(Не определен),(Не определен),585,КЛАССИКА,КОЛЬЦО,СРЕДНЯЯ,NaN,(Не определен),"1,412","1,412",NaN,КРАСНЫЙ,0,17
4,4300019591,Кольцо родир. алм.гр. 17.0,ИФБК,CLASSIC CZ NS,(Не определен),ГЕОМЕТРИЯ,(Не определен),(Не определен),585,КЛАССИКА,КОЛЬЦО,ДОРОГАЯ,NaN,(Не определен),"1,71","1,71",NaN,КРАСНЫЙ,0,17


In [8]:
inventory = pd.read_csv('/content/gdrive/My Drive/A Reinforcement Learning Approach for Inventory Optimization in Retail/DemandRestoring_v0/data/remains_4001.csv', sep=',', decimal='.')
inventory['Timestamp'] = pd.to_datetime(inventory[u'updated_at'])
inventory = inventory.set_index(['Timestamp'], drop=True)
inventory.head()

,shop_id,product_id,updated_at,quantity
Timestamp,,,,
2015-08-31,4001,4300059161,2015-08-31,0
2016-01-02,4001,4300059161,2016-01-02,0
2015-08-31,4001,4300059161,2015-08-31,1
2013-04-17,4001,4300051009,2013-04-17,1
2013-08-11,4001,4300051009,2013-08-11,0


In [9]:
product_id = 4900001067 # sku
shop_id = 4001
inventory[(inventory['product_id'] == product_id) & (inventory['shop_id'] == shop_id)]

,shop_id,product_id,updated_at,quantity
Timestamp,,,,
2015-12-12,4001,4900001067,2015-12-12,5
2015-09-03,4001,4900001067,2015-09-03,0
2015-08-31,4001,4900001067,2015-08-31,33
2016-06-05,4001,4900001067,2016-06-05,0


In [10]:
# Агрегируем на всякий случай остатки на одну дату (берём максимально известный остаток за дату)
iv_ts = inventory.reset_index().groupby('Timestamp').agg({'quantity': np.max}).rename(columns={'quantity': 'inventory'})
iv_ts.head()

,inventory
Timestamp,
2013-01-02,0
2013-01-03,0
2013-01-04,0
2013-01-05,0
2013-01-06,0


In [11]:
# дополним пропущенные значения (т.е. добавим пропущенные даты в DF и заполним значения остатков в эти даты нулями)
iv_ts = iv_ts.reindex(pd.date_range(np.min(iv_ts.index), np.max(iv_ts.index))).fillna(method = 'ffill')
iv_ts.head()

,inventory
2013-01-02,0.0
2013-01-03,0.0
2013-01-04,0.0
2013-01-05,0.0
2013-01-06,0.0


In [115]:
# Добавим данные о продажах
sales_ts = sales[(sales['product_id'] == product_id) & (sales['shop_id'] == shop_id)].rename(columns={'quantity':'sales'})

# смёржим продажи и остатки
iv_sales = sales_ts[['sales']].merge(iv_ts, how='right', left_index=True, right_index = True)
iv_sales['sales'].fillna(0, inplace=True)
iv_sales.head()

,sales,inventory
2013-01-02,0.0,0.0
2013-01-03,0.0,0.0
2013-01-04,0.0,0.0
2013-01-05,0.0,0.0
2013-01-06,0.0,0.0


In [116]:
# для более консервативной оценки удалим начало истории и "хвост" истории, когда и продажи, и остатки были равны нулю
positive_iv_sales = iv_sales[iv_sales.max(axis=1) > 0]
life_start_date = positive_iv_sales.index[0]
life_end_date = positive_iv_sales.index[-1]

iv_sales = iv_sales[(iv_sales.index >= life_start_date) & (iv_sales.index <= life_end_date)] 
iv_sales.head()

,sales,inventory
2013-01-11,0.0,2.0
2013-01-12,0.0,0.0
2013-01-13,0.0,0.0
2013-01-14,0.0,0.0
2013-01-15,5.0,0.0


In [117]:
# даты, когда и остатки, и продажи равны нулю
zero_idx = (iv_sales['inventory'] == 0) & (iv_sales['sales'] == 0)

# даты, когда продажи равны остаткам
s_eq_i_idx = (iv_sales['inventory'] == iv_sales['sales']) 

# даты, когда продажи больше или равны остаткам
s_gr_i_idx = (iv_sales['inventory'] <= iv_sales['sales'])  

# даты, когда был дефицит (out-of-stock)
oos_idx = zero_idx # подход к определнию момента дефицита может быть другим
# oos_idx = s_gr_i_idx # более мягкий вариант (больше где восстановим спрос)
iv_sales[(oos_idx)].head()

,sales,inventory
2013-01-12,0.0,0.0
2013-01-13,0.0,0.0
2013-01-14,0.0,0.0
2013-01-18,0.0,0.0
2013-01-20,0.0,0.0


In [123]:
sum_k = iv_sales['sales'].sum()
n_k_less_m = iv_sales['sales'][(~zero_idx) & (~s_gr_i_idx)].count() # кол-во дней, когда продажи строго меньше остатка
n_k_equal_m = iv_sales['sales'][(~zero_idx) & s_gr_i_idx].count() # кол-во дней, когда весь остаток был распродан

print('sum_k:', sum_k)
print('n_0:', n_k_less_m)
print('n_m:', n_k_equal_m)

sum_k: 337.0
n_0: 161
n_m: 301


In [124]:
a = 1
LAMBDA = sum_k / (n_k_less_m + a * n_k_equal_m)

print('LAMBDA:', LAMBDA)
print('MEAN:', iv_sales['sales'][~oos_idx].mean())

LAMBDA: 0.7294372294372294
MEAN: 0.7294372294372294


In [125]:
iv_sales['sales'][oos_idx].count()

929

In [126]:
# Кейс: сколько было потеряно спроса за весь период
print('Суммарно за весь период с', np.min(iv_sales.index), 'по', np.max(iv_sales.index))
print('COVERED DEMAND:', sum_k)
print('LOST DEMAND:', LAMBDA * iv_sales['sales'][oos_idx].count())
print('LOST DEMAND:', LAMBDA * iv_sales['sales'][oos_idx].count() / sum_k * 100, '%')

Суммарно за весь период с 2013-01-11 00:00:00 по 2016-10-15 00:00:00
COVERED DEMAND: 337.0
LOST DEMAND: 677.6471861471862
LOST DEMAND: 201.0822510822511 %


Рекомендация к заказу:	Recommend_Order_t =OUL-(INV+Orders_t-1), IF Inv_t<= ROL 0, otherwise


In [110]:
rec_order = [0]
order = [0]
inv = [0, 0]

for t in range(1, 2):
  if inv[t] <= ROL:
    rec_order.append(OUL - inv[t] - order[t - 1])
  else:
    rec_order.append(0)

print(rec_order)

[0, 10]


recommend_order - наша рекомендация к заказу
<br>order - объем получаемого заказа

order = F(recommend_order_t, alpha)
<br>1)order <= recommend_order_t 
<br>2)$\sum_{sku}$ E(order_{sku,t}) = alpha * E(recommend_order_{sku,t})

Home Task: Realize code for Order
<br> - order = Ceil(recommend_order * alpha)  !!! requirement 2) is not met 
<br> - order = random(cdf=’chi’, m=recommend_order_{sku,t}, sigma=)  !!! requirement 2) is not met


In [152]:
from math import ceil
from scipy.stats import chi

rec_order = 10

order = ceil(rec_order * alpha)
print('1)', order)

order = chi.rvs(df=rec_order, loc=rec_order, scale=alpha, size=5, random_state=None)
print('2)', order)

order = np.random.uniform(rec_order * alpha, rec_order, 5)
print('3)', order)

1) 3
2) [10.89122845 10.86304958 11.0501341  11.14967689 11.18242111]
3) [8.33836874 7.60577663 6.72278093 9.22263949 4.2475668 ]


In [0]:
# спрос на определенный товар в данный момент времени при остатке m
from math import exp
from math import factorial

def demand(m):
  summ = 0
  for l in range(m):
    summ += LAMBDA**l * math.exp(-LAMBDA) / math.factorial(l)
  return 1 - summ

In [142]:
demand(3)

0.037820569754640876

In [0]:
# остатки
# inv[t] = max(inv[t - 1] + order_come[t] - demand(m), 0)
# m = inv[t - 1]?

In [0]:
# продажи
# sales = max(demand(m), inv[t - 1] + order_come[t])